# Ensemble

In [ ]:
import numpy as np
import pandas as pd
import gc
import time

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

I use stacking with predictions for the validation set for all the models. Load the pedictions and combine them to the new feature matrix. Do the same for the test predictions.

In [ ]:
knn_train = pd.read_pickle('knn_train.pickle')
rf_train = pd.read_pickle('rf_train.pickle')
xgb_train = pd.read_pickle('xgb_train.pickle')
cat_train = pd.read_pickle('cat_train.pickle')
knn_test = pd.read_pickle('knn_test.pickle')
rf_test = pd.read_pickle('rf_test.pickle')
xgb_test = pd.read_pickle('xgb_test.pickle')
cat_test = pd.read_pickle('cat_test.pickle')

In [ ]:
X_train = np.column_stack((knn_train, rf_train, xgb_train, cat_train))
X_test = np.column_stack((knn_test, rf_test, xgb_test, cat_test))

In [ ]:
data = pd.read_pickle('matrix.pickle')
Y_train = data[data.date_block_num == 33]['item_cnt_month']

In [ ]:
del data
del knn_train
del rf_train
del xgb_train
del cat_train
del knn_test
del rf_test 
del xgb_test
del cat_test
gc.collect();

In [ ]:
ts = time.time()
meta_model = LinearRegression()
meta_model.fit(X_train, Y_train)
Y_test = meta_model.predict(X_test).clip(0,20)
time.time() - ts

In [ ]:
test = pd.read_csv('test.csv.gz', compression='gzip').set_index('ID')
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('ensemble_submission.csv', index=False)